In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import pandas as pd
import datetime

In [2]:
spark = SparkSession.builder.appName("myApp").getOrCreate()

data = [
    (101, "2025-01-03", 250),
    (101, "2025-01-05", 300),
    (102, "2025-01-01", 150),
    (102, "2025-01-02", 200),
    (103, "2025-01-04", 500)
]

columns = ["customer_id", "purchase_date", "amount"]

df = spark.createDataFrame(data, columns)

windowSpec = Window.partitionBy("customer_id").orderBy("purchase_date")

df = (
    df.withColumn("row_num", row_number().over(windowSpec))
      .filter(col("row_num") == 1)
      .drop("row_num")
)

df.show()


+-----------+-------------+------+
|customer_id|purchase_date|amount|
+-----------+-------------+------+
|        101|   2025-01-03|   250|
|        102|   2025-01-01|   150|
|        103|   2025-01-04|   500|
+-----------+-------------+------+



In [3]:
"""
SELECT emp_id,
       COUNT(*) - 1 AS salary_change_count
FROM (
    SELECT emp_id, salary,
           ROW_NUMBER() OVER (PARTITION BY emp_id ORDER BY effective_date) AS rn
    FROM employee_salaries
) t
GROUP BY emp_id
HAVING COUNT(DISTINCT salary) > 1;

"""

'\nSELECT emp_id, \n       COUNT(*) - 1 AS salary_change_count\nFROM (\n    SELECT emp_id, salary, \n           ROW_NUMBER() OVER (PARTITION BY emp_id ORDER BY effective_date) AS rn\n    FROM employee_salaries\n) t\nGROUP BY emp_id\nHAVING COUNT(DISTINCT salary) > 1;\n\n'